<a href="https://colab.research.google.com/github/sftSalman/transferLearning/blob/main/Transfer_learning__Cats_Dog_Classifiaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import urllib.request
import os
import shutil
import zipfile
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile

In [12]:
local_zip = '/content/drive/MyDrive/Deep Learning/tensorFlow/new_cats_dog/cats_dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Deep Learning/tensorFlow/new_cats_dog')
zip_ref.close()

In [13]:
source_path = '/content/drive/MyDrive/Deep Learning/tensorFlow/new_cats_dog/PetImages'

In [14]:

source_path_dogs = os.path.join(source_path,'Dog')
source_path_cats = os.path.join(source_path,'Cat')

In [15]:
len(os.listdir(source_path_cats))

12501

In [16]:
len(os.listdir(source_path_dogs))

12501

In [17]:
root_dir = '/content/drive/MyDrive/Deep Learning/tensorFlow/New'

In [20]:

if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

In [21]:
def create_train_test_dirs(root_path):
  os.makedirs(os.path.join(f'{root_path}/training'))
  os.makedirs(os.path.join(f'{root_path}/training','dogs'))
  os.makedirs(os.path.join(f'{root_path}/training','cats'))
  os.makedirs(os.path.join(f'{root_path}/testing'))
  os.makedirs(os.path.join(f'{root_path}/testing','dogs'))
  os.makedirs(os.path.join(f'{root_path}/testing','cats'))

In [22]:
try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [23]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/content/drive/MyDrive/Deep Learning/tensorFlow/New/training
/content/drive/MyDrive/Deep Learning/tensorFlow/New/testing
/content/drive/MyDrive/Deep Learning/tensorFlow/New/training/dogs
/content/drive/MyDrive/Deep Learning/tensorFlow/New/training/cats
/content/drive/MyDrive/Deep Learning/tensorFlow/New/testing/dogs
/content/drive/MyDrive/Deep Learning/tensorFlow/New/testing/cats


In [24]:
def split_data(source,training,testing,split_size):
  shuffled_source = random.sample(os.listdir(source),len(os.listdir(source)))
  training_number = int(len(shuffled_source)*split_size)
  

  i = 0
  target = training
  for item in shuffled_source:
    item_source = os.path.join(source,item)
    if os.path.getsize(item_source)==0:
      print(f'{item} is zero length , so ignoring')
    else :
      copyfile(item_source,os.path.join(target,item))
      i += 1



    if i == training_number:
      target = testing

In [25]:
cat_source_dir = '/content/drive/MyDrive/Deep Learning/tensorFlow/new_cats_dog/PetImages/Cat'
dog_source_dir = '/content/drive/MyDrive/Deep Learning/tensorFlow/new_cats_dog/PetImages/Dog'

training_dir = '/content/drive/MyDrive/Deep Learning/tensorFlow/New/training'
testing_dir = '/content/drive/MyDrive/Deep Learning/tensorFlow/New/testing'

training_cats_dir = os.path.join(training_dir,'cats/')
testing_cats_dir = os.path.join(testing_dir,'cats/')

training_dogs_dir = os.path.join(training_dir,'dogs/')
testing_dogs_dir = os.path.join(testing_dir,'dogs/')

if len(os.listdir(training_cats_dir)) > 0:
  for file in os.scandir(training_cats_dir):
    os.remove(file.path)
if len(os.listdir(training_dogs_dir)) > 0:
  for file in os.scandir(training_dogs_dir):
    os.remove(file.path)
if len(os.listdir(testing_cats_dir)) > 0:
  for file in os.scandir(testing_cats_dir):
    os.remove(file.path)
if len(os.listdir(testing_dogs_dir)) > 0:
  for file in os.scandir(testing_dogs_dir):
    os.remove(file.path)

split_size = .9
split_data(cat_source_dir,training_cats_dir,testing_cats_dir,split_size)
split_data(dog_source_dir, training_dogs_dir, testing_dogs_dir, split_size)

666.jpg is zero length , so ignoring
11702.jpg is zero length , so ignoring


In [26]:
print(f"\n\nThere are {len(os.listdir(training_cats_dir))} images of cats for training")
print(f"There are {len(os.listdir(training_dogs_dir))} images of dogs for training")
print(f"There are {len(os.listdir(testing_cats_dir))} images of cats for testing")
print(f"There are {len(os.listdir(testing_dogs_dir))} images of dogs for testing")



There are 11250 images of cats for training
There are 11250 images of dogs for training
There are 1250 images of cats for testing
There are 1250 images of dogs for testing


In [27]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40,
                                  
                                   width_shift_range=0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode='nearest')

In [28]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [31]:
train_generator = train_datagen.flow_from_directory(
        training_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        testing_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [32]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7faaa8e1f450>)

In [33]:
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

# Load pretrained weights
pre_trained_model.load_weights(weights_file)

# free the layers
for layer in pre_trained_model.layers:
  layer.trainable = False

# 
last_layer = pre_trained_model.get_layer("mixed7")
print("Last layer output shape: ", last_layer.output_shape)
last_output = last_layer.output

Last layer output shape:  (None, 7, 7, 768)


In [34]:
# Flatten the output into 1-D
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and Relu activation
x = layers.Dense(1024, activation="relu")(x)
# Add a final sigmoid layer for activation
x = layers.Dense(1, activation="sigmoid")(x)

model = Model(pre_trained_model.input, x)

In [35]:
model.compile(optimizer=RMSprop(lr=1e-4),
              loss="binary_crossentropy",
              metrics=["acc"])

# train the model 
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=2,
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
  33/1125 [..............................] - ETA: 19:50 - loss: 0.5292 - acc: 0.7848

KeyboardInterrupt: ignored

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()
plt.figure(figsize=(15, 30))

n_images = len(uploaded.keys())


for idx, fn in enumerate(uploaded.keys()):
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  plt.subplot(1, n_images,idx+1 )
  plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
    plt.title(fn + " is a dog")
  else:
    print(fn + " is a cat")
    plt.title(fn + " is a cat")
plt.show()